In [ ]:
#<start>only for colab users

In [1]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 31.7 MB/s 


In [2]:
!mkdir images

In [3]:
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/sajjad.jpg -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/ali.jpg -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/mohsen.jpg -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/farid.jpg -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/1.jpg -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/2.png -P images
!wget https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/mohsen1.png -P images

--2022-09-07 16:39:31--  https://github.com/sajjadranjbar73/deep-face-recognition/raw/master/images/sajjad.jpg
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sajjadranjbar73/deep-face-recognition/master/images/sajjad.jpg [following]
--2022-09-07 16:39:31--  https://raw.githubusercontent.com/sajjadranjbar73/deep-face-recognition/master/images/sajjad.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36802 (36K) [image/jpeg]
Saving to: ‘images/sajjad.jpg’

sajjad.jpg          100%[===================>]  35.94K  --.-KB/s    in 0s      

2022-09-07 16:39:31 (179 MB/s) - ‘images/sajjad.jpg’ saved [

In [7]:
!wget https://raw.githubusercontent.com/sajjadranjbar73/deep-face-recognition/master/ArcFace.py

--2022-09-07 16:46:21--  https://raw.githubusercontent.com/sajjadranjbar73/deep-face-recognition/master/ArcFace.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4043 (3.9K) [text/plain]
Saving to: ‘ArcFace.py.1’

ArcFace.py.1        100%[===================>]   3.95K  --.-KB/s    in 0s      

2022-09-07 16:46:21 (56.2 MB/s) - ‘ArcFace.py.1’ saved [4043/4043]



In [ ]:
#<end>only for colab users

In [8]:
import ArcFace
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
%matplotlib inline

In [9]:
model = ArcFace.loadModel()
face_detector = MTCNN()

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://drive.google.com/uc?id=1LVB3CdVejpmGHM28BpqqkbZP5hDEcdZY
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:02<00:00, 67.8MB/s]


In [10]:
def detect_face(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #mtcnn expects RGB but OpenCV read BGR
    detections = face_detector.detect_faces(img_rgb)
    detection = detections[0]
    x, y, w, h = detection["box"]
    detected_face = img[int(y):int(y+h), int(x):int(x+w)]
    return detected_face

In [11]:
def preprocess_face(img, target_size=(112,112)):
    img = cv2.imread(img)
    img = detect_face(img)
    img = cv2.resize(img, target_size)
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]
    return img_pixels

In [12]:
def img_to_encoding(path):
    img = preprocess_face(path)
    return model.predict(img)[0]

In [19]:
database = {}

database["sajjad"] = img_to_encoding("./images/sajjad.jpg")
database["ali"] = img_to_encoding("./images/ali.jpg")
database["mohsen"] = img_to_encoding("./images/mohsen.jpg")
database["farid"] = img_to_encoding("./images/farid.jpg")

In [14]:
def EuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [15]:
verification_threshhold = 4.4

## Face Verification

In [16]:
def verify(image_path, identity, database):
    # Step 1: Compute the encoding for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path) 
    
    # Step 2: Compute distance with identity's image
    dist = EuclideanDistance(encoding, database[identity])
    
    # Step 3: Open the door if dist < verification_threshhold, else don't open
    if dist < verification_threshhold:
        print("It's " + str(identity) + ", welcome!")
    else:
        print("It's not " + str(identity) + ", please go away")
             
    return dist

In [20]:
verify("images/1.jpg", "sajjad", database)

It's sajjad, welcome!


3.579871

In [22]:
verify("images/ali.jpg", "sajjad", database)

It's not sajjad, please go away


4.9722075

## Face Identification

In [23]:
def who_is_it(image_path, database):
   
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding()
    encoding = img_to_encoding(image_path)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 
    min_dist = 1000
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = EuclideanDistance(encoding, db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if min_dist > dist:
            min_dist = dist
            identity = name

  
    if min_dist > verification_threshhold:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [ ]:
who_is_it("images/1.png", database)

In [ ]:
who_is_it("images/mohsen1.jpg", database)

it's mohsen, the distance is 4.3644657


(4.3644657, 'mohsen')